<a href="https://colab.research.google.com/github/murafba/Learn-Pandas/blob/main/Exercise_3_Renaming_and_Combining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D06f4f46b266ecbbcbf466f00bedb49dc8cf7bbac079187c564347cafb91cee23e45e4a01d78d87810189211c580595d14f8950739427080889d552ffd34188512602bc02624e0a8e6c3cb0c57b2fa3297372acf37f727339fda2ad31a9f53aba87b83026df34bea12e824eebf5ca84cdaecfcef9e0eb5b094ec40a9a402bccec2d6764b5277baaf3a81edfca88c10b71c183521581c80f73a12c550628de1db24336873eb342ee026909b1b1557ad362df2718f073d5092e79795e8c4e0c66d4c1b6ea3b2764230d75f0a8ccf735afa359399db07d02c4c045385830d57abb8b48637be7002c7af80961fefc21bd35c95c2a45a272ff9390da0f5b5d17cdd383,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D750f9936958843bb0e1f4cde78f20fa265c590cdffe20456db644ea14a2f09f3757eef420214975ee319474d0c2b503945a281282d86aabb4787b4f5d790a716e2f548f90293dd1c0bb54b38c2443deb9392549f0081325dae361001d519720409b5652bd5023c6916564cb72cd77ce6ca277f2e5a9015d213ac3d5d8b7c5421e962d9823f7458f0368a9f555f81ba92025ae906aa54204a94b82c94a2e4178b7cb27f1768964315741bbc78d511930a507e288a01c8aa459ae607d80c4075bbb8d0d7593eb283f1159cf8f5d08cd883ff30fe76bd49b045e6adb6ca73edce88435ff86ceb0eeeba7fafe9ba390f6e179f22f01d8f44fa93167afaa5cd0af58d,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D237f366155f129c30442f12635ec5221c4f64a7b3da9dc4878c8a901e1ed107ba29480e29140da82308ac5cda084e8bc5d9cb53e3d53487396f1badf0f4d3171eba7a8ec40027bd3b51203dada8d66305416191e23aecb212c926baa185ac7788a369ffe8a54dcf4e96163adaad9f616a29e74564385b140c4059e38944b5c1ae792fd38be90017798da9501b76daceaa2feb7b8fde48fc696f2c1c478cebb50157fe323dc87e543467218eab300a0259fd0caf1d3ec107740c37d66f610802c5725bd0b5539452b1dff6e452b8325bd69c5f06a6d26e47045f16a5a991df8144b4e3caf34a96fb8b3ad4e8f03cc3a4555ef4c16b7312a4f9bdb2533b47e3ca1,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D65b553b07709544a6e7b4d50c7d0c0344a05590e4600f4e75c88e0b3a2aa2fe54a7dad9469a4773ac53db1967b558960a8b8528ad3d0ad8ab71ecc2699c3d760b46f63c56ea345441f4aabe9cee715852e4e301cf154f78b948041b45c4b7d694ab1e5a2ccc263922a2a8de34e2d6896894442e89072891700ade8b0f66723561a5b4c07d59c94455eff76534263e043a657fecf13bcbe5264327843ccd719c6a4c876dd508a067ed24fb69309e33cdb8e41ab3aad7b790e393b9eeef28b22c44cbb4c4fb5dc239313a66a732d8314c4c535af8bb10f51a3ca695f00f9734a55d65335dd144d9baf0679479c69575efad4fa9fe1ad800fbdc5201cdbf316a746,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0549de0a4d0124d90af51bc02498210e041a2a80932dc59ec872f3a079589ee1c43d15acfb5104efad3ae2876e77ccea9b310a24ed33775ed6cf73b21f33701aa0eda2bf4d8e37930779323e40e0b76bdb1b745b08a3447500ae7dc68fe924949cdf47a8e6ed8ab2bec7851713e103cb3ec05a4e4d847c1c1e50daa6505882d6d2e7014aa07a0f7a608dff51f106be85ee0bf1c630ea4fa967781461748cdab9c65e32a7dc40670fca9b7e3ec71100794fe1786e93ea8bce1afc3768e7111b2070b781917b2dad3e23a3d80cea97f7ce64a4bdc4d8eea9866684f5a4db35e6b76fef5d96bcc16ec3f5fac9f1d1313d51567ab4a3589d4b48871d41c25d43e856,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D860c9170fd3713965daea929481fabe9fbb7a46968537794bcff2c7b4db35cd67a62e3059c320ff86fc919c491a67c10b26ba358d4b5b9ba2c563c0edbf3317fac10b4816a0fb934dbe004f078b68ea984bc21282bb54d0bb359f7ef4276038434ea427d59ce48241a46ab77082b571c203eb66c36938a9fd35b7efd939512220ef6bc1b7237943fec9c997de6ebd49343346fb2f9e9a080a931c74dffe83feb2118d4b3ae4a28674ea5b872f7fc3a6d8b3ddfd273649694abf3a526bdfc926067eb2ac574dea68a6bfd89363a37bb9e5adcb936ddf342f6ab23099abc25d79ffa604d99841cd29ffcad39f5e4b81f388c623b10d2134491b05c3ccf98a91522,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D48963341ab24215e37c9628103a57656613fb7bef2d4e3ea03762cd6784fc496309b3df4f056849ffd3b48695bb454adbe0cfbf69c506ba9f95a4ceea9b4306565fa34028373ada96691713dbbb244544fa3a5da6228202053dc1abdd8fee2f651a93d528d428d0062b7f089cc81740fe6f1bcf5fce88d0f966a7e75e41989b82efebc079b996e44c8f661fb22d4b0f4d8a93f4807802583bbb160f351bbc517aa4defb70c0533b022bc555f21a4fd5e122969a2217daa6a163359c39314f7e55ddc2d74a475b52f9d18426037868beb3b702608989a4ee2dd972f7f855f2e655c85d365e7b2d2b4739a69a27e7bcfb0988734509330a501d59a0aca10181aea,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D04ea9b7288da12851562afac1f7999772e2494ef4cc2b214502c81b12334966e9da267808f3b6b810963870a5b7fb86c7ef41e89a6e132fc7b3ac46e5969cb2ca20ec8fc3a90d8b619d05a7d6b38b50068575244bc955b5eaf9ca9e3a9c3d589f0b786bdd767956c5bc3855e2d823fe58ae7251b1f4fdc341ff33dc25da8212d586da5f5d9b1521b24657434b57cabca38cc85648e3d534aa1a33a0ada32774baaa72c2857e1f3d858ad8161a74bd1296bac90d5430e011ee5fdc2da2237a66321618d5fafba44f851000c049a0bbcf581fe9f2f57f36133490b289c3b28cde2ea77057bee1dbb02880add9b34222aaeee87bc8e276b848073067a05bbe34ee7,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D784a7f210ee60dee0fc94b05300fde115e86bd6f799c3e7d1a32c5216d23ab8e6d8fd835898d3eaab7b0a0b1b8661990c24d1e8fce318d048549b5534c2a18e2b4af50e8d3e32ff5d19a53bb951b7eaf78b6b4b1f2ef3a40ec7f758b225e9c933272ab08315ab72aefb90ad81d9f1659e36e3604f67b7977efc337d3f7904bae18a1e76bc41187c82757fb93e5095979305c8ac6c23c204c2ff30177b3a37eeccdc72656761065fd8de5243a8485b2effec737fa45075305d688d5237bc5d2c3fb4f1050a815a7e4ff68334d1ec270de56a56130c89b9cdfaa91245262bd4026e4677e50e6ccf4c55e05c9136e3b6b7eedee535fa0934a0f0de9ce9a25a68877,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T153650Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dadbc218d33a780fea48235c91613d697591f0af73c98785be4a0dd184100a80c5b4d9e7173e0f54223b4a8cb67e02fe7cbee12e34a5b44812d9e47bba0c9405730c4ff92e43b42ee528b677e32bce3a299f0f07b0f8a6cb52c5538a83297431b8d3c4de1503d198498d7ee93844064f6572f34720ef6e499588d266dfc861ddfdc2a46b3a185303e546b306c923edc2b0fef82ed9346eb3161349638f144f30c9b312d1a67fc9c41c90831bd1069b3736f8ca85b1afbc150f251783d5a66c5967ba8c66214023551fadd0a3ce51907fdb87f4ff3f89e782e8fc1d306d0a1fff29ab6923af0fe647032ff15b5c0c4c34a09e9dbe255c7f7cec95575c448ef13b5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/renaming-and-combining).**

---


# Introduction

Run the following cell to load your data and some utility functions.

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.renaming_and_combining import *
print("Setup complete.")

# Exercises

View the first several lines of your data by running the cell below:

In [ ]:
reviews.head()

## 1.
`region_1` and `region_2` are pretty uninformative names for locale columns in the dataset. Create a copy of `reviews` with these columns renamed to `region` and `locale`, respectively.

In [ ]:
# Your code here
renamed = reviews.rename(columns={'region_1': 'region', 'region_2': 'locale'})

# Check your answer
q1.check()

In [ ]:
#q1.hint()
#q1.solution()

## 2.
Set the index name in the dataset to `wines`.

In [ ]:
reindexed = reviews.rename_axis("wines", axis='columns')

# Check your answer
q2.check()

reindexed

In [ ]:
#q2.hint()
#q2.solution()

## 3.
The [Things on Reddit](https://www.kaggle.com/residentmario/things-on-reddit/data) dataset includes product links from a selection of top-ranked forums ("subreddits") on reddit.com. Run the cell below to load a dataframe of products mentioned on the */r/gaming* subreddit and another dataframe for products mentioned on the *r//movies* subreddit.

In [ ]:
gaming_products = pd.read_csv("../input/things-on-reddit/top-things/top-things/reddits/g/gaming.csv")
gaming_products['subreddit'] = "r/gaming"
movie_products = pd.read_csv("../input/things-on-reddit/top-things/top-things/reddits/m/movies.csv")
movie_products['subreddit'] = "r/movies"
# type(movie_products['subreddit'])

Create a `DataFrame` of products mentioned on *either* subreddit.

In [ ]:
combined_products = pd.concat([gaming_products, movie_products])

# Check your answer
q3.check()

In [ ]:
#q3.hint()
#q3.solution()

## 4.
The [Powerlifting Database](https://www.kaggle.com/open-powerlifting/powerlifting-database) dataset on Kaggle includes one CSV table for powerlifting meets and a separate one for powerlifting competitors. Run the cell below to load these datasets into dataframes:

In [ ]:
powerlifting_meets = pd.read_csv("../input/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("../input/powerlifting-database/openpowerlifting.csv")
pd.set_option('display.max_rows', 5)
# powerlifting_meets
powerlifting_competitors

In [ ]:
powerlifting_meets

Both tables include references to a `MeetID`, a unique key for each meet (competition) included in the database. Using this, generate a dataset combining the two tables into one.

In [ ]:
powerlifting_combined = powerlifting_competitors.set_index('MeetID').join(powerlifting_meets.set_index('MeetID'))

# Check your answer
q4.check()

In [ ]:
powerlifting_combined

In [ ]:
#q4.hint()
#q4.solution()

# Congratulations!

You've finished the Pandas micro-course.  Many data scientists feel efficiency with Pandas is the most useful and practical skill they have, because it allows you to progress quickly in any project you have.

If you'd like to apply your new skills to examining geospatial data, you're encouraged to check out our **[Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis)** micro-course.

You can also take advantage of your Pandas skills by entering a **[Kaggle Competition](https://www.kaggle.com/competitions)** or by answering a question you find interesting using **[Kaggle Datasets](https://www.kaggle.com/datasets)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*